In [ ]:
#Module Imports
import numpy as np
import numpy.random as npr
import math
import matplotlib.pyplot as plt
import scipy as sp
from scipy import stats
import xlwings as xw

In [ ]:
'''DEFINITION OF VARIABLES
    S0 - Stock Price at T=0
    E - Strike Price
    T - Time in Years
    R - Risk Free Rate
    SIGMA - Volatility
    DT - Time Step = T/N
    DF - Discount Factor = e^-RT
    I - Number of Simulations
'''

S0 = 100
Barrier = 80
Optionality = "In"   #Out = Knock Out Option   In = Knock In Option
Type = "P"  #C = Call  P = Put

E=100
T=1
R=0.05
SIGMA=0.20
I = 1000
N=252

In [ ]:
'''OPTION VALUATION - MONTE CARLO SIMULATION W/ ANTITHETIC VARIANCE REDUCTION W/ MILSTEIN SCHEME '''
def option_valuation(S0, E, T, N, SIGMA, R, I, P):    
    DT = T/N   #Time Step   
#GENERATE RANDOM NUMBERS - ANTITHETIC VARIANCE REDUCTION
    PHI = npr.standard_normal((N,int(I/2))) 
    PHI = np.concatenate((PHI, -PHI), axis=1)     
#SET UP EMPTY ARRAYS AND SET INITIAL VALUES    
    S = np.zeros_like(PHI)  #Array to Capture Asset Value Path
    S[0] = S0
    
#CREATE FOR LOOP TO GENERATE SIMULATION PATHS - MILSTEIN METHOD
    for t in range (1,N):
        S[t]=S[t-1]*(1+R*DT+(SIGMA*PHI[t]*np.sqrt(DT))+(SIGMA**2)*(0.5*(((PHI[t]**2)-1)*DT)))
        
    return S

In [ ]:
#Run full simulation
S = option_valuation(S0, E, T, N, SIGMA, R, I, P)
S[1,1]
#You can un-comment the xw.view to get the sim results in excel
#xw.view(S)

In [ ]:
#Evaluation of up and out call
#If at any point in the simulation path the underlying value does reach the barrier, the value of the path is zero.
if Type == "C" and Optionality == "Out":
    for t in range(0,I-1):
        max = np.max(S[:,t])
        if max > Barrier:
            S[:,t]=0

#Evaluation of down and out put
#If at any point in the simulation path the underlying value does reach the barrier, the value of the path is zero.
if Type == "P" and Optionality == "Out":
    for t in range(0,I-1):
        min = np.min(S[:,t])
        if min < Barrier:
            S[:,t]=0

#Evaluation of up and in call
#If at any point in the simulation path the underlying value does NOT reach the barrier, the value of the path is zero.
if Type == "C" and Optionality == "In":
    for t in range(0,I-1):
        max = np.max(S[:,t])
        if max < Barrier:
            S[:,t]=0

#Evaluation of down and in put
#If at any point in the simulation path the underlying value does NOT reach the barrier, the value of the path is zero.
if Type == "P" and Optionality == "In":
    for t in range(0,I-1):
        min = np.min(S[:,t])
        if min > Barrier:
            S[:,t]= 0 

#You can un-comment the xw.view to get the sim results (after adjusting for barrier) in excel
#xw.view(S)

In [ ]:
%matplotlib inline
#Calculation of Discounted Expected Payoff for Options

DF = math.exp(-R*T)  #Discount Factor

print("Number of Simulations %.d" %I)

if Type == "C":
    Call_Value = DF * np.sum(np.maximum(S[-1] - E, 0)) / I
    print( "Value of Call Option Monte Carlo = %.3f" %Call_Value)

if Type == "P":
    Put_Value = np.maximum(E - S[-1], 0)
    Put_Value[Put_Value == E] = 0
    Put_Value = DF * np.sum(Put_Value) / I
    
    print( "Value of Put Option Monte Carlo = %.3f" %Put_Value)

#Create Graph of Monte Carlo Simulation
plt.plot(S)
plt.show()